In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from mytest import get_mnist_pair, dual_sinkhorn
from metrics import WassersteinApproximation, CostMatrixType

In [ ]:
image, noised_image = get_mnist_pair()
image = image.unsqueeze(0)
noised_image = noised_image.unsqueeze(0)

regularization = 5

ref_wass = dual_sinkhorn(image, noised_image, regularization)
my_wass = WassersteinApproximation(
    regularization=regularization,
    cost_matrix_type=CostMatrixType.L2
)(image, noised_image)

print(f'Wasserstein:{ref_wass};MyWasserstein:{my_wass}')